In [1]:
from bs4 import BeautifulSoup
import re # import Regular expression operations module
import requests
from time import gmtime, strftime

In [2]:
a = []
url_str = input("Enter the product category you want to search for: ")
url = []

Enter the product category you want to search for: Electronics


In [4]:
# Hacky fix
words = url_str.split()
var = len(words)

In [5]:
if var == 1:
	url = "https://www.amazon.com/s/ref=nb_sb_noss_1?url=search-alias%3Daps&field-keywords=" + words[0]

if var == 2:
	url = "https://www.amazon.com/s/ref=nb_sb_noss_1?url=search-alias%3Daps&field-keywords=" + words[0] + "+" + words[1]

elif var == 3:
	url = "https://www.amazon.com/s/ref=nb_sb_noss_1?url=search-alias%3Daps&field-keywords=" + words[0] + "+" + words[1] + "+" + words[2]


In [6]:
# Add header
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}

In [7]:
r = requests.get(url, headers=headers)

In [ ]:
# print(r) # print request to see if Response 200

In [8]:
soup = BeautifulSoup(r.content, "html.parser")

In [9]:
# Csv writing setup
filename = "products.csv"
f = open(filename, "w", encoding='utf-8')

In [10]:
# Grab time and format string 
strftime("%Y-%m-%d %H:%M:%S", gmtime())

'2020-12-20 17:38:05'

In [11]:
headers ="Asin, Name," + "Price : " + strftime("%Y-%m-%d %H:%M:%S", gmtime()) + ", Number of Reviews\n"
f.write(headers)

58

In [ ]:
# Regex if needed
# a = re.compile((?<=data-asin))
# Used this to beautify and inspect the html/xml data ----> http://jsbeautifier.org/

In [12]:
# Style 1 of Amazon's product display
containers1 = soup.findAll("li", {"class":"s-result-item s-result-card-for-container a-declarative celwidget "})
print("containers style 1: ", len(containers1))

containers style 1:  0


In [13]:
#page could be styled different, invoke query second style
containers2 = soup.findAll("li", {"class":"s-result-item s-result-card-for-container s-carded-grid celwidget "})
print("containers style 2: ", len(containers2))

containers style 2:  0


In [14]:
#check for sponsored containers
sponsored_containers = soup.findAll("li", {"class":"s-result-item celwidget AdHolder"}) 
print("containers style 3 sponsored: ", len(sponsored_containers))

containers style 3 sponsored:  0


In [15]:
#check for the most common style
common_containers = soup.findAll("li", {"class":"s-result-item celwidget "})  
print("containers style 4 common: ", len(common_containers))

containers style 4 common:  0


In [16]:
#check for special style
containers3 = soup.findAll("li", {"class":"s-result-item s-col-span-12 celwidget "})
print("containers style 5 special", len(containers3))

containers style 5 special 0


In [17]:
for container in sponsored_containers:
	# Product Asin
	asin = (container["data-asin"])

	# Product Name
	try:
		title_container = container.findAll("a", {"class":"a-link-normal s-access-detail-page s-color-twister-title-link a-text-normal"}) 
		name = title_container[0]["title"]
	except:
		name = "N/A"

	# Product Price # span class="a-offscreen"
	price_container = container.findAll("span", {"class":"a-offscreen"})
	price = price_container[1].text

	#Number of reviews
	num_review_container = container.findAll("a", {"class":"a-size-small a-link-normal a-text-normal"})
	try:
		if (len(num_review_container) > 1):
			num_reviews = num_review_container[1].text
		else: 
			num_reviews = num_review_container[0].text 
	except:
		num_reviews = "0"

	f.write(asin + ',' + name.replace(",", "|") + ',' + price.replace("$", "") + "," + num_reviews.replace(",", "") + "\n") # + ',' + name.replace(",", "|") + ',' + price + "," + num_reviews + "\n")


for container in common_containers:
	# Product Asin
	asin = (container["data-asin"])

	# Product Name
	try:
		title_container = container.findAll("a", {"class":"a-link-normal s-access-detail-page s-color-twister-title-link a-text-normal"}) 
		name = title_container[0]["title"]
	except:
		name = "N/A"

	# Product Price # span class="a-offscreen"
	price_container = container.findAll("span", {"class":"a-offscreen"})
	# print(price_container)
	try:
		price = price_container[0].text
	except:
		price = "N/A"

	#Number of reviews
	num_review_container = container.findAll("a", {"class":"a-size-small a-link-normal a-text-normal"})
	try:
		if (len(num_review_container) > 1):
			num_reviews = num_review_container[1].text
		else:
			num_reviews = num_review_container[0].text
	except:
		num_reviews = "0"

# try:
# num_review2 = num_review_container[1].text.strip()
# print(num_review2)
# except list index out of range:
# num_review2 = 0;

	f.write(asin + ',' + name.replace(",", "|") + ',' + price.replace("$", "") + "," + num_reviews.replace(",", "") + "\n")

In [18]:
f.close()